# Regressão Linear usando Validação Cruzada

Avaliar um modelo de Machine Learning pode ser bastante complicado. Normalmente, dividimos o conjunto de dados em conjuntos de treinamento e teste e usamos o conjunto de treinamento para treinar o modelo e o conjunto de testes para testar o modelo. Em seguida, avaliamos o desempenho do modelo com base em uma métrica de erro para determinar a precisão do modelo. Esse método, no entanto, não é muito confiável, pois a precisão obtida para um conjunto de testes pode ser muito diferente da precisão obtida para um conjunto de testes diferente. A validação cruzada  fornece uma solução para esse problema dividindo os dados em seções e garantindo que cada seção seja usada como um conjunto de testes em algum momento.



 <img src='../grid_search_workflow.png' />

# O que é a Validação Cruzada com k Seções (k Fold CV)?

O K-Fold CV é o local em que um determinado conjunto de dados é dividido em um número K de seções / dobras, onde cada dobra é usada como um conjunto de testes em algum momento. Vamos pegar o cenário de validação cruzada de 5 vezes (K = 5). Aqui, o conjunto de dados é dividido em 5 dobras. Na primeira iteração, a primeira dobra é usada para testar o modelo e o restante é usado para treinar o modelo. Na segunda iteração, a segunda dobra é usada como o conjunto de testes, enquanto o restante serve como o conjunto de treinamento. Esse processo é repetido até que cada dobra das 5 dobras tenha sido usada como conjunto de teste.

 <img src='../k_fold.png' />

# **Vamos começar!**
Utilizaremos o mesmo Dataset do passo a passo sobre Regressão Linear ***'USA_Housing.csv'***

## Importando as bibliotecas

In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


## Carregando os dados

In [111]:
USAhousing = pd.read_csv('USA_Housing.csv')
USAhousing.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701..."
1,79248.642455,6.002900,6.730821,3.09,40173.072174,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA..."
2,61287.067179,5.865890,8.512727,5.13,36882.159400,1.058988e+06,"9127 Elizabeth Stravenue\nDanieltown, WI 06482..."
3,63345.240046,7.188236,5.586729,3.26,34310.242831,1.260617e+06,USS Barnett\nFPO AP 44820
4,59982.197226,5.040555,7.839388,4.23,26354.109472,6.309435e+05,USNS Raymond\nFPO AE 09386


## As variáveis preditoras e a variável de resposta

Para treinar o modelo de regressão, primeiro precisaremos dividir nossos dados em uma matriz **X** que contenha os dados das variáveis preditoras e uma matriz **y** com os dados da variável de destino, nesse caso a coluna Preço. Vamos descartar a coluna Endereço, pois ela possui apenas informações de texto que o modelo de regressão linear não pode usar.

### Matrizes X e y

In [112]:
X = USAhousing[['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms',
               'Avg. Area Number of Bedrooms', 'Area Population']]
y = USAhousing['Price']

## Separando os dados de treinamento e de validação

In [131]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

## Regressão Linear dos mínimos quadrados ordinários

In [143]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [144]:
coeff_df = pd.DataFrame(lm.coef_,X.columns,columns=['Coefficient'])
coeff_df

,Coefficient
Avg. Area Income,21.642680
Avg. Area House Age,165227.889797
Avg. Area Number of Rooms,121886.318611
Avg. Area Number of Bedrooms,1446.725402
Area Population,15.240921


## Cross Validation ou Validação Cruzada

Faremos uma validação cruzada para verificar esse comportamento. Os dados serão divididos em 5 grupos.


In [174]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [175]:
scores = []
best_lm = LinearRegression()
kf = KFold(n_splits=5, random_state=30, shuffle=False)

In [188]:
print ("Tamanho de X:",len(X), "\n")

for train_index, test_index in kf.split(X):
    #print("Train Index: ", train_index,len(train_index))
    #print("Test Index: ", test_index, len(test_index),"\n")
    
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    best_lm.fit(X_train,y_train)
    predictions = best_lm.predict(X_test)
     
    MAE = metrics.mean_absolute_error(y_test, predictions)
    print("MAE: ",MAE)
    print('MAE_NORMALIZADO:', MAE/y.mean())
    
    resultado = cross_val_score(best_lm, X_test, y_test, cv = 5)
    print("SCORE: ",resultado.mean())
    
    coeff_df = pd.DataFrame(best_lm.coef_,X.columns,columns=['Coefficient'])
    print(coeff_df,"\n\n\n\n\n ")
    

Tamanho de X: 5000 

MAE:  80458.23930459908
MAE_NORMALIZADO: 0.0653031613306974
SCORE:  0.9160164588216835
                                Coefficient
Avg. Area Income                  21.575051
Avg. Area House Age           165483.252702
Avg. Area Number of Rooms     120234.313874
Avg. Area Number of Bedrooms    2863.642681
Area Population                   15.300839 




 
MAE:  84256.0981781121
MAE_NORMALIZADO: 0.06838565720522576
SCORE:  0.919673338169388
                                Coefficient
Avg. Area Income                  21.428819
Avg. Area House Age           166641.425789
Avg. Area Number of Rooms     121840.997622
Avg. Area Number of Bedrooms    1375.214987
Area Population                   15.119496 




 
MAE:  79821.80016120586
MAE_NORMALIZADO: 0.0647866015797335
SCORE:  0.9147436580304806
                                Coefficient
Avg. Area Income                  21.650729
Avg. Area House Age           164980.478518
Avg. Area Number of Rooms     120975.577348
A

Agora, vamos aplicar a MinMax scaling técnica de pré-processamento para normalizar o conjunto de dados.

In [27]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)